# Azure ML - Sample Batch Prediction Pipeline

This notebook demonstrates creation and execution of an Azure ML pipeline designed to load data from a remote source, to make predictions against that data using a previously registered ML model, and finally save that data  

### Import Required Packages

In [1]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.22.0 to work with mm-oshkosh-aml


In [2]:
import os
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

parent = os.path.dirname(directory_path)
parent_folder_name = os.path.basename(parent)
print("My user directory name is: " + parent_folder_name)

user = parent_folder_name
user = parent_folder_name
user = user.replace('_', '')
user = user.replace('-', '')
user = user[:10]
print('user=' + user)
experiment_name = parent_folder_name + '-004-code-a-thon-diabetes-pipeline-parquet'

My current directory is : /mnt/batch/tasks/shared/LS_root/mounts/clusters/mm-compute/code/Users/memasanz/code-a-thon
My directory name is : code-a-thon
My user directory name is: memasanz
user=memasanz


In [3]:
from azureml.core import Dataset

#use default datastore
default_ds = ws.get_default_datastore()

if user + '-diabetes-tabular-dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.parquet'], # Upload the diabetes csv files in /data
                        target_path= user + '-diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)  
    parquet_paths = [(default_ds, user + '-diabetes-data/*.parquet')]
    tab_data_set  = Dataset.Tabular.from_parquet_files(path=parquet_paths)

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name= user + '-diabetes-tabular-dataset',
                                description='diabetes data',
                                tags = {'format':'parquet'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [4]:
import pandas as pd
raw_dataset = pd.read_parquet('./data/diabetes.parquet')
raw_dataset = raw_dataset.drop(['PatientID', 'Diabetic'], axis = 1)
raw_dataset = raw_dataset.head(25)
raw_dataset.to_parquet('./data/diabetes-raw.parquet', index = False)


if user + '-diabetes-tabular-dataset-raw' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes-raw.parquet'], # Upload the diabetes csv files in /data
                        target_path= user + '-diabetes-data-raw/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    parquet_paths = [(default_ds, user + '-diabetes-data-raw/diabetes-raw.parquet')]
    tab_data_set  = Dataset.Tabular.from_parquet_files(path=parquet_paths)

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name= user + '-diabetes-tabular-dataset-raw',
                                description='diabetes data',
                                tags = {'format':'parquet'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Uploading an estimated of 1 files
Uploading ./data/diabetes-raw.parquet
Uploaded ./data/diabetes-raw.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered.


In [5]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, user + '-004-batch-inferencing-pipeline')

#Get default datastore
default_ds = ws.get_default_datastore()

### Create Compute Resources

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = user + "-cluster"
cluster_name = cluster_name[-16:]
print('trying to create: ' + cluster_name)

try:
    # Check for existing compute target
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2, idle_seconds_before_scaledown=1800)
        compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
        compute_target.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

trying to create: memasanz-cluster
Found existing cluster, use it.


## Run Configuration
Create configuration for the running pipeline.  The RunConfiguration defines the environment used in the python steps

In [9]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies



run_config = RunConfiguration()
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                                                            pip_packages=['azureml-sdk','numpy', 'joblib', 'sklearn', 'pyarrow'])


## Define Output Datasets

Below are the configuration for datasets that will be passed between steps in our pipelien.  Note, in all cases we specifiy the datastore that should hold the datasets and wheather they should be registered following step completion or not.  This can optionally be disabled by removing the register_on_complete() call


In [11]:
inference_outputfile_name = user + '_004_diabetes_inferencing_dataset'
print(inference_outputfile_name)
inference_dataset_name    = user + '_004_pipeline_diabetes_inferencing_data'
print(inference_dataset_name)
scored_outputfile_name = user + '_004_diabetes_scored_dataset'
print(scored_outputfile_name)
scored_dataset_name    = user + '_004_pipeline_diabetes_scored_data'
print(scored_dataset_name)

inferencing_dataset = OutputFileDatasetConfig(name=inference_outputfile_name, destination=(default_ds, user + 'inferencing_dataset/{run-id}')).read_parquet_files().register_on_complete(name= inference_dataset_name)
scored_dataset      = OutputFileDatasetConfig(name=scored_outputfile_name, destination=(default_ds, user + 'scored_dataset/{run-id}')).read_parquet_files().register_on_complete(name=scored_dataset_name)


memasanz_004_diabetes_inferencing_dataset
memasanz_004_pipeline_diabetes_inferencing_data
memasanz_004_diabetes_scored_dataset
memasanz_004_pipeline_diabetes_scored_data


# Define Pipeline Parameters

PipelineParameter objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Multiple pipeline parameters can be created and used. Included here are multiple sample pipeline parameters (get_data_param_*) to highlight how parameters can be passed into and consumed by various pipeline steps.

In [12]:
model_name_parm  = PipelineParameter(name='model_name', default_value= user + '-diabetes_code-a-thon-model')
user_param       = PipelineParameter(name='user_param', default_value= user)
get_data_param_2 = PipelineParameter(name='get_data_param_2', default_value='value_2')
get_data_param_3 = PipelineParameter(name='get_data_param_3', default_value='value_3')

In [13]:
import os, shutil
folder_name = 'batch-inferencing'
script_folder = os.path.join(os.getcwd(), folder_name)
print(script_folder)
os.makedirs(script_folder, exist_ok=True)


/mnt/batch/tasks/shared/LS_root/mounts/clusters/mm-compute/code/Users/memasanz/code-a-thon/batch-inferencing


In [14]:
%%writefile $script_folder/get_inferencing_data.py

  
from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse


#Parse input arguments
parser = argparse.ArgumentParser("Get Inferencing Data")
parser.add_argument('--user_param', type=str, required=True)
parser.add_argument('--get_data_param_2', type=str, required=True)
parser.add_argument('--get_data_param_3', type=str, required=True)
parser.add_argument('--inferencing_dataset', dest='inferencing_dataset', required=True)

# Note: the get_data_param args below are included only as an example of argument passing.
# They are not consumed in the code sample shown here.
args, _ = parser.parse_known_args()

user_param = args.user_param
get_data_param_2 = args.get_data_param_2
get_data_param_3 = args.get_data_param_3
inferencing_dataset = args.inferencing_dataset

print(str(type(inferencing_dataset)))
print(inferencing_dataset)

#Get current run
current_run = Run.get_context()

#Get associated AML workspace
ws = current_run.experiment.workspace

#Get default datastore
ds = ws.get_default_datastore()

dataset = Dataset.get_by_name(name =user_param + '-diabetes-tabular-dataset-raw', workspace = ws)
df = dataset.to_pandas_dataframe()

print('dataset shape = ' + str(df.shape))
print('saving inferencing data: ' + inferencing_dataset)

# Save dataset for consumption in next pipeline step
os.makedirs(inferencing_dataset, exist_ok=True)
#df.to_csv(os.path.join(inferencing_dataset, 'inferencing_data.csv'), index=False)


# Save dataset for consumption in next pipeline step
# os.makedirs(inferencing_dataset, exist_ok=True)
df.to_parquet(os.path.join(inferencing_dataset, 'inferencing_data.parquet'), index=False)



Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/mm-compute/code/Users/memasanz/code-a-thon/batch-inferencing/get_inferencing_data.py


In [15]:
%%writefile $script_folder/score_inferencing_data.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.core.model import Model
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse
import joblib
import json
import joblib
import numpy as np
from azureml.core.model import Model
import time
import pandas as pd
import azureml.core
from azureml.core import Workspace, Dataset
import os
import math


# Parse input arguments
parser = argparse.ArgumentParser("Score Inferencing Data")
parser.add_argument('--user_param', type=str, required=True)
parser.add_argument('--model_name_parm', type=str, required=True)
parser.add_argument('--scored_dataset', dest='scored_dataset', required=True)

args, _ = parser.parse_known_args()
user = args.user_param
model_name = args.model_name_parm
scored_dataset = args.scored_dataset

# Get current run
current_run = Run.get_context()

# Get associated AML workspace
ws = current_run.experiment.workspace

# Get default datastore
ds = ws.get_default_datastore()


inferencing_dataset = current_run.input_datasets[user + '_004_diabetes_inferencing_data']
inferencing_data_df = inferencing_dataset.to_pandas_dataframe()


print('inferencing data df shape:' + str(inferencing_data_df.shape))

print('model_name' + model_name)

# Get model from workspace - the code below will always retrieve the latest version of the model; specific versions can be targeted.
model_list = Model.list(ws, name=model_name, latest=True)
model_path = model_list[0].download(exist_ok=True)
model = joblib.load(model_path)


print(inferencing_data_df.shape)


# Make predictions with new dataframe
predictions = model.predict(inferencing_data_df)

print('made predictions')

print(predictions)


inferencing_data_df['Predictions']=predictions

print(inferencing_data_df.head(5))


# Save scored dataset
os.makedirs(scored_dataset, exist_ok=True)
print(scored_dataset)

os.makedirs(scored_dataset, exist_ok=True)
print(scored_dataset)
#inferencing_data_df.to_csv(os.path.join(scored_dataset, 'scored_data.csv'), index=False)


inferencing_data_df.to_parquet(os.path.join(scored_dataset, 'scored_data.parquet'), index=False)
# Save scored dataset



Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/mm-compute/code/Users/memasanz/code-a-thon/batch-inferencing/score_inferencing_data.py


In [16]:
%%writefile $script_folder/publish_scored_data.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse


# Parse input arguments
parser = argparse.ArgumentParser("Score Inferencing Data")
parser.add_argument('--user_param', type=str, required=True)


args, _ = parser.parse_known_args()
user = args.user_param



# Get current run
current_run = Run.get_context()

# Get associated AML workspace
ws = current_run.experiment.workspace

# Get default datastore
ds = ws.get_default_datastore()

# Get inferencing dataset
scored_dataset = current_run.input_datasets[user + '_004_diabetes_scored_data']
scored_data_df = scored_dataset.to_pandas_dataframe()

# Save dataset to ./outputs dir
os.makedirs('./outputs', exist_ok=True)
#scored_data_df.to_csv(os.path.join('outputs', 'scored_data.csv'), index=False)


# # Save dataset to ./outputs dir
os.makedirs('./outputs', exist_ok=True)
scored_data_df.to_parquet(os.path.join('outputs', 'scored_data.parquet'), index=False)



Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/mm-compute/code/Users/memasanz/code-a-thon/batch-inferencing/publish_scored_data.py


In [17]:
get_inferencing_data_step = PythonScriptStep(
    name='Get Inferencing Data',
    script_name='get_inferencing_data.py',
    arguments=[
        '--user_param', user_param,
        '--get_data_param_2', get_data_param_2,
        '--get_data_param_3', get_data_param_3,
        '--inferencing_dataset', inferencing_dataset
    ],
    outputs=[inferencing_dataset],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

score_inferencing_data_step = PythonScriptStep(
    name='Score Inferencing Data',
    script_name='score_inferencing_data.py',
    arguments=[
        '--user_param', user_param,
        '--model_name_parm', model_name_parm,
        '--scored_dataset', scored_dataset
    ],
    inputs=[inferencing_dataset.as_input(name=user + '_004_diabetes_inferencing_data')],
    outputs=[scored_dataset],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

publish_scored_data_step = PythonScriptStep(
    name='Publish Scored Data',
    script_name='publish_scored_data.py',
    arguments=[
        '--user_param', user_param,
    ],
    inputs=[scored_dataset.as_input(name=user + '_004_diabetes_scored_data')],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

# Create Pipeline

Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [18]:
pipeline = Pipeline(workspace=ws, steps=[get_inferencing_data_step, score_inferencing_data_step, publish_scored_data_step])

In [19]:
experiment = Experiment(ws, user + '-004-exp_batch_predictions')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

Created step Get Inferencing Data [7f6b597e][c4a85009-5d7d-48d1-a491-e44fd314b5c6], (This step will run and generate new outputs)
Created step Score Inferencing Data [ab03ede6][d06add0d-d862-4820-8353-c522bda9af5e], (This step will run and generate new outputs)
Created step Publish Scored Data [fe70a8cc][fc8b616e-a1e7-4151-bdf2-49a5f7f18413], (This step will run and generate new outputs)
Submitted PipelineRun 494c98fa-9f43-43b8-a1cd-9032b5240bd0
Link to Azure Machine Learning Portal: https://studio.ml.azure.us/experiments/memasanz-004-exp_batch_predictions/runs/494c98fa-9f43-43b8-a1cd-9032b5240bd0?wsid=/subscriptions/c778c451-6f3b-48e1-86b1-e55fa7615154/resourcegroups/mm-oshkosh-aml-rg/workspaces/mm-oshkosh-aml
PipelineRunId: 494c98fa-9f43-43b8-a1cd-9032b5240bd0
Link to Azure Machine Learning Portal: https://studio.ml.azure.us/experiments/memasanz-004-exp_batch_predictions/runs/494c98fa-9f43-43b8-a1cd-9032b5240bd0?wsid=/subscriptions/c778c451-6f3b-48e1-86b1-e55fa7615154/resourcegroups/

fc2b9863058f: Pushed
0c7d6f1219f0: Pushed
1a1a19626b20: Pushed
5b7dc8292d9b: Pushed
e342e7653d41: Pushed
bbc674332e2e: Pushed
088c5f014be3: Pushed
da2785b7bb16: Pushed
8547178b7afb: Pushed
e14bec5d61b3: Pushed
1: digest: sha256:c4b15d65da719194127d9049f33c070f83403778b5805714570662f3cb6222ab size: 4721
2021/11/29 17:37:23 Successfully pushed image: mmoshkoshcontainerregistry.azurecr.us/azureml/azureml_fcd79c16ce634dd20c038668584a7029:1
2021/11/29 17:37:23 Executing step ID: acb_step_2. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/11/29 17:37:23 Pushing image: mmoshkoshcontainerregistry.azurecr.us/azureml/azureml_fcd79c16ce634dd20c038668584a7029:latest, attempt 1
The push refers to repository [mmoshkoshcontainerregistry.azurecr.us/azureml/azureml_fcd79c16ce634dd20c038668584a7029]
aeecfd30e3f1: Preparing
285089fe2d75: Preparing
3e02b5e191b4: Preparing
987ebe99f9d9: Preparing
e14bec5d61b3: Preparing
e6dead1dd63a: Preparing
d51d632f8341: Preparing
5056bd07


Streaming azureml-logs/65_job_prep-tvmps_c9e85b794165304cfdf20469ad0fefd2e1b7b40a31a661303b31b0360d0ffa28_d.txt
[2021-11-29T17:41:42.473555] Entering job preparation.
[2021-11-29T17:41:43.083179] Starting job preparation.
[2021-11-29T17:41:43.083213] Extracting the control code.
[2021-11-29T17:41:43.083807] Starting extract_project.
[2021-11-29T17:41:43.083855] Starting to extract zip file.
[2021-11-29T17:41:43.102557] Finished extracting zip file.
[2021-11-29T17:41:43.106379] Using urllib.request Python 3.0 or later
[2021-11-29T17:41:43.106465] Start fetching snapshots.
[2021-11-29T17:41:43.106533] Start fetching snapshot.
[2021-11-29T17:41:43.106554] Retrieving project from snapshot: b632e583-d43b-447f-bdee-d71bb18f4460
Starting the daemon thread to refresh tokens in background for process with pid = 46
[2021-11-29T17:41:43.361829] Finished fetching snapshot.
[2021-11-29T17:41:43.361869] Finished fetching snapshots.
[2021-11-29T17:41:43.361889] Finished extract_project.
[2021-11-29T


StepRun(Get Inferencing Data) Execution Summary
StepRun( Get Inferencing Data ) Status: Finished
{'runId': 'd564075e-6353-4f38-97b5-c51f2d30c579', 'target': 'memasanz-cluster', 'status': 'Completed', 'startTimeUtc': '2021-11-29T17:41:04.910838Z', 'endTimeUtc': '2021-11-29T17:42:58.709583Z', 'properties': {'ContentSnapshotId': 'b632e583-d43b-447f-bdee-d71bb18f4460', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.moduleid': 'c4a85009-5d7d-48d1-a491-e44fd314b5c6', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '7f6b597e', 'azureml.pipelinerunid': '494c98fa-9f43-43b8-a1cd-9032b5240bd0', 'azureml.pipeline': '494c98fa-9f43-43b8-a1cd-9032b5240bd0', 'azureml.pipelineComponent': 'masterescloud', '_azureml.ComputeTargetType': 'amlcompute', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [{'dataset': {'id': 'e5a1d264-6aa4-44a4-a470-ad88bd1b32d6'}, 'consumptionDetails': {'type':




StepRunId: 9d03e695-a322-457d-8b52-861b7c5f8e5c
Link to Azure Machine Learning Portal: https://studio.ml.azure.us/experiments/memasanz-004-exp_batch_predictions/runs/9d03e695-a322-457d-8b52-861b7c5f8e5c?wsid=/subscriptions/c778c451-6f3b-48e1-86b1-e55fa7615154/resourcegroups/mm-oshkosh-aml-rg/workspaces/mm-oshkosh-aml
StepRun( Score Inferencing Data ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_c9e85b794165304cfdf20469ad0fefd2e1b7b40a31a661303b31b0360d0ffa28_d.txt
2021-11-29T17:43:07Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9d03e695-a322-457d-8b52-861b7c5f8e5c/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9d03e695-a322-457d-8b52-861b7c5f8e5c/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=17915 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9d03e


Streaming azureml-logs/70_driver_log.txt
2021/11/29 17:43:22 Got JobInfoJson from env
2021/11/29 17:43:22 Starting App Insight Logger for task:  runTaskLet
2021/11/29 17:43:22 Version: 3.0.01769.0004 Branch: 2021-11-05 Commit: a04a71b
2021/11/29 17:43:22 Attempt 1 of http call to http://10.0.0.4:16384/sendlogstoartifacts/info
2021/11/29 17:43:22 Send process info logs to master server succeeded
2021/11/29 17:43:22 Attempt 1 of http call to http://10.0.0.4:16384/sendlogstoartifacts/status
2021/11/29 17:43:22 Send process info logs to master server succeeded
[2021-11-29T17:43:22.599766] Entering context manager injector.
[2021-11-29T17:43:23.105295] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'Dataset:context_managers.Datasets', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['score_inferencing_data.py', '--user_param', 'memasanz', '--model_name_parm',




StepRunId: 9a5c1b0f-cb15-4659-a287-76251ed511de
Link to Azure Machine Learning Portal: https://studio.ml.azure.us/experiments/memasanz-004-exp_batch_predictions/runs/9a5c1b0f-cb15-4659-a287-76251ed511de?wsid=/subscriptions/c778c451-6f3b-48e1-86b1-e55fa7615154/resourcegroups/mm-oshkosh-aml-rg/workspaces/mm-oshkosh-aml
StepRun( Publish Scored Data ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_c9e85b794165304cfdf20469ad0fefd2e1b7b40a31a661303b31b0360d0ffa28_d.txt
2021-11-29T17:43:58Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9a5c1b0f-cb15-4659-a287-76251ed511de/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9a5c1b0f-cb15-4659-a287-76251ed511de/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=17915 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/mm-oshkosh-aml/azureml/9a5c1b0f


Streaming azureml-logs/75_job_post-tvmps_c9e85b794165304cfdf20469ad0fefd2e1b7b40a31a661303b31b0360d0ffa28_d.txt
[2021-11-29T17:44:15.732647] Entering job release
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 6.0.1 (/azureml-envs/azureml_86a06bacfe696e757c50aa0dceb63621/lib/python3.6/site-packages), Requirement.parse('pyarrow<2.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
[2021-11-29T17:44:16.744327] Starting job release
[2021-11-29T17:44:16.744853] Logging experiment finalizing status in history service.
[2021-11-29T17:44:16.745074] job release stage : upload_datastore starting...
Starting the daemon thread to refresh tokens in background for process with pid = 337
[2021-11-29T17:44:16.745414] job release stage : start importing azureml.history._tracking in run_history_release.
[2021-11-29T17:44:16.746123] job release stage : copy_batchai_cached_logs starting...
[2021-11-29



PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '494c98fa-9f43-43b8-a1cd-9032b5240bd0', 'status': 'Completed', 'startTimeUtc': '2021-11-29T17:30:18.700406Z', 'endTimeUtc': '2021-11-29T17:44:25.923722Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{"user_param":"memasanz","get_data_param_2":"value_2","get_data_param_3":"value_3","model_name":"memasanz-diabetes_code-a-thon-model"}', 'azureml.continue_on_step_failure': 'False', 'azureml.pipelineComponent': 'pipelinerun'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://mmoshkoshaml3050074004.blob.core.usgovcloudapi.net/azureml/ExperimentRun/dcid.494c98fa-9f43-43b8-a1cd-9032b5240bd0/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=4T%2FETZ76UfhTOljlxxSTe%2Byx2F3kY%2FLdtOD%2BsCnIgKM%3D&skoid=dcb2fb7d-d2b5-4f11-9766-eea1f247b606&sktid=150e302b-984e-491f-a34e-758d0390a174&skt=2021-11-29T16%

'Finished'

# Publish Pipeline

Create a published version of pipeline that can be triggered via a REST API call

In [20]:
published_pipeline = pipeline.publish(name = user + '004 Batch Prediction Pipeline',
                                     description = 'Pipeline that generates batch predictions using a registered trained model.',
                                     continue_on_step_failure = False)

In [ ]:
published_pipeline

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": user + "rest-api-diabetes-batch"})
run_id = response.json()["Id"]
run_id

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments[user + "rest-api-diabetes-batch"], run_id)

# Block until the run completes
published_pipeline_run.wait_for_completion(show_output=True)